In [2]:
!pip install scikit-fuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894077 sha256=9575860a64fa29c737ad1ca1734cbb0258833a50177264e6e45973556df0d6df
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


In [3]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot, plot
init_notebook_mode(connected=True)
import plotly.express as px
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [4]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [7]:
datatest = pd.read_csv("/content/datatest.csv")
datatest2 = pd.read_csv("/content/datatest2.csv")
datatraining = pd.read_csv("/content/datatraining.csv")

In [8]:
scaler = MinMaxScaler()
columns = ['Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio']
scaler.fit(np.array(datatraining[columns]))
datatest[columns] = scaler.transform(np.array(datatest[columns]))
datatest2[columns] = scaler.transform(np.array(datatest2[columns]))
datatraining[columns] = scaler.transform(np.array(datatraining[columns]))

In [9]:
datatest['date'] = pd.to_datetime(datatest['date'])
datatest2['date'] = pd.to_datetime(datatest2['date'])
datatraining['date'] = pd.to_datetime(datatraining['date'])
datatest.reset_index(drop=True, inplace=True)
datatest2.reset_index(drop=True, inplace=True)
datatraining.reset_index(drop=True, inplace=True)

In [10]:
datatraining.describe()

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
count,8143.000000,8143.000000,8143.000000,8143.000000,8143.000000,8143.000000
mean,0.387341,0.401676,0.077292,0.119942,0.312576,0.212330
std,0.243281,0.247233,0.125947,0.194536,0.224186,0.408982
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.167464,0.154431,0.000000,0.016246,0.106304,0.000000
50%,0.332536,0.423623,0.000000,0.025220,0.296338,0.000000
75%,0.571770,0.616307,0.165795,0.139925,0.441308,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
scaler = MinMaxScaler()
columns = ['Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio']
scaler.fit(np.array(datatraining[columns]))
datatest[columns] = scaler.transform(np.array(datatest[columns]))
datatest2[columns] = scaler.transform(np.array(datatest2[columns]))
datatraining[columns] = scaler.transform(np.array(datatraining[columns]))

In [12]:
hours_1 = []
hours_0 = []
for date in datatraining[datatraining['Occupancy'] == 1]['date']:
    hours_1.append(date.hour)
for date in datatraining[datatraining['Occupancy'] == 0]['date']:
    hours_0.append(date.hour)

In [13]:
datatest['period_of_day'] = [1 if (i.hour >= 7 and i.hour <= 17) else 0 for i in datatest['date']]
datatest2['period_of_day'] = [1 if (i.hour >= 7 and i.hour <= 17) else 0 for i in datatest2['date']]
datatraining['period_of_day'] = [1 if (i.hour >= 7 and i.hour <= 17) else 0 for i in datatraining['date']]
datatraining.sample(10)

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy,period_of_day
7284,2015-02-09 19:14:59,0.452153,0.860655,0.000000,0.503946,0.745535,0,0
1279,2015-02-05 15:10:00,0.837321,0.516035,0.332076,0.427820,0.551534,1,1
5586,2015-02-08 14:57:00,0.308612,0.423958,0.040095,0.005931,0.311298,0,1
200,2015-02-04 21:10:59,0.598086,0.418147,0.000000,0.054000,0.384777,0,0
558,2015-02-05 03:08:59,0.460925,0.364063,0.000000,0.019341,0.297927,0,0
3608,2015-02-07 05:58:59,0.167464,0.118226,0.000000,0.018103,0.018498,0,0
3172,2015-02-06 22:43:00,0.239234,0.093642,0.000000,0.017484,0.011162,0,0
2592,2015-02-06 13:02:59,0.604067,0.145491,0.031365,0.192326,0.130447,0,1
1744,2015-02-05 22:54:59,0.358852,0.199128,0.000000,0.020579,0.126492,0,0
392,2015-02-05 00:23:00,0.547847,0.377919,0.000000,0.025220,0.333498,0,0


In [15]:
df = datatraining

In [16]:
temperature = ctrl.Antecedent(np.arange(df['Temperature'].min(), df['Temperature'].max(), 1), 'temperature')
humidity = ctrl.Antecedent(np.arange(df['Humidity'].min(), df['Humidity'].max(), 1), 'humidity')
light = ctrl.Antecedent(np.arange(df['Light'].min(), df['Light'].max(), 1), 'light')
co2 = ctrl.Antecedent(np.arange(df['CO2'].min(), df['CO2'].max(), 1), 'co2')

occupancy = ctrl.Consequent(np.arange(df['Occupancy'].min(), df['Occupancy'].max(), 1), 'occupancy')

In [ ]:
humidity['low'] = fuzz.trimf(humidity.universe, [df['Humidity'].min(), df['Humidity'].min(), 30])
humidity['medium'] = fuzz.trimf(humidity.universe, [25, 30, 40])
humidity['high'] = fuzz.trimf(humidity.universe, [30, df['Humidity'].max(), df['Humidity'].max()])


In [ ]:
light['low'] = fuzz.trimf(light.universe, [df['Light'].min(), df['Light'].min(), 300])
light['medium'] = fuzz.trimf(light.universe, [200, 300, 600])
light['high'] = fuzz.trimf(light.universe, [300, df['Light'].max(), df['Light'].max()])

In [ ]:
co2['low'] = fuzz.trimf(co2.universe, [df['CO2'].min(), df['CO2'].min(), 800])
co2['medium'] = fuzz.trimf(co2.universe, [600, 800, 1200])
co2['high'] = fuzz.trimf(co2.universe, [800, df['CO2'].max(), df['CO2'].max()])

In [ ]:
occupancy['no'] = fuzz.trimf(occupancy.universe, [df['Occupancy'].min(), df['Occupancy'].min(), 0.5])
occupancy['yes'] = fuzz.trimf(occupancy.universe, [0.5, df['Occupancy'].max(), df['Occupancy'].max()])


In [ ]:
# Update membership functions
temperature['low'] = fuzz.gaussmf(temperature.universe, df['Temperature'].min(), 2)
temperature['medium'] = fuzz.gaussmf(temperature.universe, df['Temperature'].mean(), 2)
temperature['high'] = fuzz.gaussmf(temperature.universe, df['Temperature'].max(), 2)

humidity['low'] = fuzz.gaussmf(humidity.universe, df['Humidity'].min(), 2)
humidity['medium'] = fuzz.gaussmf(humidity.universe, df['Humidity'].mean(), 2)
humidity['high'] = fuzz.gaussmf(humidity.universe, df['Humidity'].max(), 2)

light['low'] = fuzz.gaussmf(light.universe, df['Light'].min(), 50)
light['medium'] = fuzz.gaussmf(light.universe, df['Light'].mean(), 50)
light['high'] = fuzz.gaussmf(light.universe, df['Light'].max(), 50)

co2['low'] = fuzz.gaussmf(co2.universe, df['CO2'].min(), 100)
co2['medium'] = fuzz.gaussmf(co2.universe, df['CO2'].mean(), 100)
co2['high'] = fuzz.gaussmf(co2.universe, df['CO2'].max(), 100)


In [ ]:
# Update fuzzy rules
rule1 = ctrl.Rule(temperature['low'] & humidity['low'] & light['low'] & co2['low'], occupancy['no'])
rule2 = ctrl.Rule(temperature['medium'] & humidity['medium'] & light['medium'] & co2['medium'], occupancy['yes'])
rule3 = ctrl.Rule(temperature['high'] & humidity['high'] & light['high'] & co2['high'], occupancy['yes'])

In [ ]:
occupancy_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])

In [ ]:
occupancy_simulation = ctrl.ControlSystemSimulation(occupancy_ctrl)

In [ ]:
# Apply the fuzzy logic to the dataset
for index, row in df.iterrows():
    occupancy_simulation.input['temperature'] = row['Temperature']
    occupancy_simulation.input['humidity'] = row['Humidity']
    occupancy_simulation.input['light'] = row['Light']
    occupancy_simulation.input['co2'] = row['CO2']

    # Compute the result
    occupancy_simulation.compute()

    # Update the dataframe with the predicted occupancy
    df.at[index, 'Predicted_Occupancy'] = occupancy_simulation.output['occupancy']


In [ ]:
for index, row in df.iterrows():
    # Update temperature membership functions based on a simple moving average
    temperature['low'].mf[0] = 0.8 * temperature['low'].mf[0] + 0.2 * row['Temperature']
    temperature['medium'].mf[1] = 0.8 * temperature['medium'].mf[1] + 0.2 * row['Temperature']
    temperature['high'].mf[2] = 0.8 * temperature['high'].mf[2] + 0.2 * row['Temperature']

    # Set the current temperature value for simulation
    occupancy_simulation.input['temperature'] = row['Temperature']

    # Compute the result
    occupancy_simulation.compute()

    # Update the dataframe with the predicted occupancy
    df.at[index, 'Predicted_Occupancy'] = occupancy_simulation.output['occupancy']


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming 'Occupancy' and 'Predicted_Occupancy' columns are present in your DataFrame
y_true = df['Occupancy']
y_pred = df['Predicted_Occupancy'].round().astype(int)  # Assuming Predicted_Occupancy is a continuous value

# Compute accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.2%}')


Accuracy: 78.77%


In [ ]:
alpha = 0.1  # Smoothing factor

for index, row in df.iterrows():
    # Update temperature membership functions using exponential moving average
    temperature['low'].mf[0] = (1 - alpha) * temperature['low'].mf[0] + alpha * row['Temperature']
    temperature['medium'].mf[1] = (1 - alpha) * temperature['medium'].mf[1] + alpha * row['Temperature']
    temperature['high'].mf[2] = (1 - alpha) * temperature['high'].mf[2] + alpha * row['Temperature']

    # Set the current temperature value for simulation
    occupancy_simulation.input['temperature'] = row['Temperature']

    # Compute the result
    occupancy_simulation.compute()

    # Update the dataframe with the predicted occupancy
    df.at[index, 'Predicted_Occupancy'] = occupancy_simulation.output['occupancy']


In [ ]:
import pandas as pd
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [1]:
learning_rate = 0.1  # Adjust the learning rate

temperature['low'] = fuzz.gaussmf(temperature.universe, df['Temperature'].min(), 2)
temperature['medium'] = fuzz.gaussmf(temperature.universe, df['Temperature'].mean(), 2)
temperature['high'] = fuzz.gaussmf(temperature.universe, df['Temperature'].max(), 2)


for index, row in df.iterrows():
    # Set the current temperature value for simulation
    occupancy_simulation.input['temperature'] = row['Temperature']

    # Compute the result
    occupancy_simulation.compute()

    # Update membership functions based on prediction error
    prediction_error = row['Occupancy'] - occupancy_simulation.output['occupancy']

    # Update membership functions for temperature based on prediction error
    temperature['low'].mf[0] = max(df['Temperature'].min(), min(df['Temperature'].max(), temperature['low'].mf[0] + learning_rate * prediction_error))
    temperature['medium'].mf[1] = max(df['Temperature'].min(), min(df['Temperature'].max(), temperature['medium'].mf[1] + learning_rate * prediction_error))
    temperature['high'].mf[2] = max(df['Temperature'].min(), min(df['Temperature'].max(), temperature['high'].mf[2] + learning_rate * prediction_error))

    # Update the dataframe with the predicted occupancy
    df.at[index, 'Predicted_Occupancy'] = occupancy_simulation.output['occupancy']

# Evaluate accuracy
y_true = df['Occupancy']
y_pred = df['Predicted_Occupancy'].round().astype(int)
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.2%}')


Accuracy: 81.34%


In [ ]:
!pip install pyspark


In [ ]:
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import FloatType
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [ ]:
# Create a Spark session
spark = SparkSession.builder.appName("FuzzyLogic").getOrCreate()

# Read data from CSV file into a Spark DataFrame
csv_file_path = "/content/datatraining.csv"
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)


In [ ]:
# Define fuzzy variables and membership functions
temperature = ctrl.Antecedent(np.arange(df.agg({"Temperature": "min"}).collect()[0][0],
                                           df.agg({"Temperature": "max"}).collect()[0][0],
                                           1), 'temperature')
humidity = ctrl.Antecedent(np.arange(df.agg({"Humidity": "min"}).collect()[0][0],
                                        df.agg({"Humidity": "max"}).collect()[0][0],
                                        1), 'humidity')
light = ctrl.Antecedent(np.arange(df.agg({"Light": "min"}).collect()[0][0],
                                    df.agg({"Light": "max"}).collect()[0][0],
                                    1), 'light')
co2 = ctrl.Antecedent(np.arange(df.agg({"CO2": "min"}).collect()[0][0],
                                  df.agg({"CO2": "max"}).collect()[0][0],
                                  1), 'co2')

occupancy = ctrl.Consequent(np.arange(0, 1, 1), 'occupancy')  # Adjust the range based on your requirements


In [ ]:
# Update membership functions for 'humidity' and other variables
humidity['low'] = fuzz.gaussmf(humidity.universe, df.agg({"Humidity": "min"}).collect()[0][0], 2)
humidity['medium'] = fuzz.gaussmf(humidity.universe, df.agg({"Humidity": "mean"}).collect()[0][0], 2)
humidity['high'] = fuzz.gaussmf(humidity.universe, df.agg({"Humidity": "max"}).collect()[0][0], 2)

